<a href="https://colab.research.google.com/github/BhargavRE25/DataScience-Portfolio/blob/master/Optimize_CNN_Practice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install keras-tuner

     |████████████████████████████████| 61kB 1.9MB/s 
  Created wheel for keras-tuner: filename=keras_tuner-1.0.1-cp36-none-any.whl size=73200 sha256=94612b2db237abaec4072db029a2c190971001682cb1d3cc8304c481042b1465
  Stored in directory: /root/.cache/pip/wheels/b9/cc/62/52716b70dd90f3db12519233c3a93a5360bc672da1a10ded43
  Created wheel for terminaltables: filename=terminaltables-3.1.0-cp36-none-any.whl size=15356 sha256=accafb96cb97b78c6b5b51c3b41a5383f758163c63f2f7a0ff888e1e8f0992d1
  Stored in directory: /root/.cache/pip/wheels/30/6b/50/6c75775b681fb36cdfac7f19799888ef9d8813aff9e379663e
Successfully built keras-tuner terminaltables


In [3]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [3]:
print(tf.__version__)

2.2.0


In [4]:
fashion_mnist = keras.datasets.fashion_mnist

In [5]:
(train_images,train_labels),(test_images,test_labels)=fashion_mnist.load_data()

4423680/4422102 [==============================] - 0s 0us/step


As the images are of Gray-Scale type, we apply scaling on these images to bring it to the range between 0 and 1 so that it will be easier for computations.

In [6]:
train_images = train_images/255.0
test_images = test_images/255.0

In [7]:
train_images[0]

array([[0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.    

In [8]:
train_images.shape

(60000, 28, 28)

In [9]:
train_images[0].shape

(28, 28)

In [10]:
train_images = train_images.reshape(len(train_images),28,28,1)
test_images = test_images.reshape(len(test_images), 28,28,1)

Creat a function to build the model for tuning your hyperparameters

In [11]:
def build_model(hp):  
  model = keras.Sequential([
    keras.layers.Conv2D(
        filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
        kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
        activation='relu',
        input_shape=(28,28,1)
    ),
    keras.layers.Conv2D(
        filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
        kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
        activation='relu'
    ),
    keras.layers.Flatten(),
    keras.layers.Dense(
        units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
        activation='relu'
    ),
    keras.layers.Dense(10, activation='softmax')
  ])
  
  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
  
  return model

In [12]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

In [13]:
tuner_search = RandomSearch(build_model, objective='val_accuracy', max_trials=5,
                            directory='output', project_name = "Mnist_Fashion")

In [14]:
tuner_search.search(train_images,train_labels,epochs=3,validation_split=0.1)

Epoch 1/3
1688/1688 [==============================] - 16s 9ms/step - loss: 0.4987 - accuracy: 0.8318 - val_loss: 0.3765 - val_accuracy: 0.8665
Epoch 2/3
1688/1688 [==============================] - 15s 9ms/step - loss: 0.3570 - accuracy: 0.8680 - val_loss: 0.3637 - val_accuracy: 0.8728
Epoch 3/3
1688/1688 [==============================] - 15s 9ms/step - loss: 0.3213 - accuracy: 0.8804 - val_loss: 0.3827 - val_accuracy: 0.8617


Epoch 1/3
1688/1688 [==============================] - 15s 9ms/step - loss: 0.4231 - accuracy: 0.8475 - val_loss: 0.3209 - val_accuracy: 0.8840
Epoch 2/3
1688/1688 [==============================] - 14s 9ms/step - loss: 0.2786 - accuracy: 0.8982 - val_loss: 0.2658 - val_accuracy: 0.9015
Epoch 3/3
1688/1688 [==============================] - 14s 9ms/step - loss: 0.2287 - accuracy: 0.9155 - val_loss: 0.2725 - val_accuracy: 0.8997


Epoch 1/3
1688/1688 [==============================] - 17s 10ms/step - loss: 0.3717 - accuracy: 0.8662 - val_loss: 0.2835 - val_accuracy: 0.8952
Epoch 2/3
1688/1688 [==============================] - 18s 10ms/step - loss: 0.2349 - accuracy: 0.9145 - val_loss: 0.2481 - val_accuracy: 0.9083
Epoch 3/3
1688/1688 [==============================] - 18s 10ms/step - loss: 0.1715 - accuracy: 0.9358 - val_loss: 0.2419 - val_accuracy: 0.9128


Epoch 1/3
1688/1688 [==============================] - 15s 9ms/step - loss: 0.3990 - accuracy: 0.8549 - val_loss: 0.3048 - val_accuracy: 0.8865
Epoch 2/3
1688/1688 [==============================] - 14s 8ms/step - loss: 0.2588 - accuracy: 0.9040 - val_loss: 0.2562 - val_accuracy: 0.9060
Epoch 3/3
1688/1688 [==============================] - 14s 8ms/step - loss: 0.2054 - accuracy: 0.9232 - val_loss: 0.2595 - val_accuracy: 0.9097


Epoch 1/3
1688/1688 [==============================] - 19s 11ms/step - loss: 0.3755 - accuracy: 0.8651 - val_loss: 0.2888 - val_accuracy: 0.9003
Epoch 2/3
1688/1688 [==============================] - 18s 10ms/step - loss: 0.2354 - accuracy: 0.9133 - val_loss: 0.2336 - val_accuracy: 0.9122
Epoch 3/3
1688/1688 [==============================] - 17s 10ms/step - loss: 0.1741 - accuracy: 0.9357 - val_loss: 0.2293 - val_accuracy: 0.9142


INFO:tensorflow:Oracle triggered exit


In [15]:
model=tuner_search.get_best_models(num_models=1)[0]

In [17]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 96)        960       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 24, 24, 64)        55360     
_________________________________________________________________
flatten (Flatten)            (None, 36864)             0         
_________________________________________________________________
dense (Dense)                (None, 64)                2359360   
_________________________________________________________________
dense_1 (Dense)              (None, 10)                650       
Total params: 2,416,330
Trainable params: 2,416,330
Non-trainable params: 0
_________________________________________________________________


In [18]:
model.fit(train_images, train_labels, epochs=10, validation_split=0.1, initial_epoch=3)

Epoch 4/10
1688/1688 [==============================] - 18s 10ms/step - loss: 0.1233 - accuracy: 0.9543 - val_loss: 0.2407 - val_accuracy: 0.9193
Epoch 5/10
1688/1688 [==============================] - 18s 11ms/step - loss: 0.0833 - accuracy: 0.9686 - val_loss: 0.2871 - val_accuracy: 0.9155
Epoch 6/10
1688/1688 [==============================] - 18s 11ms/step - loss: 0.0557 - accuracy: 0.9791 - val_loss: 0.3417 - val_accuracy: 0.9123
Epoch 7/10
1688/1688 [==============================] - 18s 11ms/step - loss: 0.0424 - accuracy: 0.9844 - val_loss: 0.3854 - val_accuracy: 0.9185
Epoch 8/10
1688/1688 [==============================] - 18s 11ms/step - loss: 0.0305 - accuracy: 0.9891 - val_loss: 0.4372 - val_accuracy: 0.9187
Epoch 9/10
1688/1688 [==============================] - 18s 10ms/step - loss: 0.0293 - accuracy: 0.9901 - val_loss: 0.4859 - val_accuracy: 0.9160
Epoch 10/10
1688/1688 [==============================] - 18s 10ms/step - loss: 0.0211 - accuracy: 0.9924 - val_loss: 0.5130 